# Part a) SGD


In [ ]:
#import numpy as np
import autograd.numpy as np
import pandas as pd
from imageio import imread
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
import os
from common import *
from models import own_LinRegGD
import cv2
from sklearn.metrics import mean_squared_error as MSE
from autograd import grad
from sklearn.linear_model import SGDRegressor


print(f"Root directory: {os.getcwd()}")
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
    "font.size": 10,
})

#%matplotlib

# Global variables

In [ ]:
SEED_VALUE = 70707070
np.random.seed(SEED_VALUE)
SAVE_FIGURES = False

# Reading data and resizing

In [ ]:
# Load the terrain
terrain1_file = "SRTM_data_Norway_1.tif"
terrain2_file = "SRTM_data_Norway_2.tif"
terrain1 =  imread(f'{INPUT_DATA}{terrain1_file}')
terrain2 = imread(f'{INPUT_DATA}{terrain2_file}')

# Resizing the image
rescale_factor = 0.1
y_size = int(terrain1.shape[0] * rescale_factor)
x_size = int(terrain1.shape[1] * rescale_factor)
terrain1Resized = cv2.resize(terrain1, (x_size, y_size))
terrain2Resized = cv2.resize(terrain2, (x_size, y_size))

# Plotting terrain
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.title.set_text("Terrain over Norway 1 (Resized)")
ax1.set_xlabel("X"); ax1.set_ylabel("Y")
surf1 = ax1.imshow(terrain1Resized, cmap='gray')
ax2.title.set_text("Terrain over Norway 2 (Resized)")
ax2.set_xlabel("X"); ax2.set_ylabel("Y")
surf2 = ax2.imshow(terrain2Resized, cmap='gray')

if SAVE_FIGURES:
    plt.savefig(f"{REPORT_FIGURES}{EX_A}terrain_data_resized.pdf")
plt.show()

# Creating image patches and Terrain data selection

In [ ]:
nXpatches = 3; nYpatches=6
y_steps = int(terrain2Resized.shape[0] / nYpatches); print(y_steps)
x_steps = int(terrain2Resized.shape[1] / nXpatches); print(x_steps)

patches_1 = create_img_patches(terrain1Resized, y_steps, x_steps)
if SAVE_FIGURES:
    fig1 = plotTerrainPatches(patches_1, nYpatches, nXpatches, plotTitle="Terrain1 patches")
    plt.savefig(f"{REPORT_FIGURES}{EX_A}Terrain1_patches.pdf")
    plt.show()

patches_2 = create_img_patches(terrain2Resized, y_steps, x_steps)
if SAVE_FIGURES:
    fig2 = plotTerrainPatches(patches_2, nYpatches, nXpatches, plotTitle="Terrain2 patches")
    plt.savefig(f"{REPORT_FIGURES}{EX_A}Terrain2_patches.pdf")
    plt.show()

# Choosing two interesting terrain patches
img1 = patches_1[2]
img2 = patches_2[5]
x1, y1, z1 = createTerrainData(img1)
x2, y2, z2 = createTerrainData(img2)

# Constructing the terrain data
terrain_data = 1
if terrain_data == 1: # Choosing terrain1*
    x, y, z = x1, y1, z1.copy() 
    #z_min = np.min(z)
    z_max = np.max(z)
    z = z1

elif terrain_data == 2: # Choosing terrain2
    x, y, z = x2, y2, z2.copy() 
    #z_min = np.min(z)
    z_max = np.max(z)
    z = z2
    
z_flat = z.ravel(); z_flat = z_flat.reshape(-1,1)

# Cost function, with autograd

In [ ]:
def cost_MSE(X,y,theta, lmb=0):
    return ((y - X @ theta)**2).sum() + lmb*(theta**2).sum()

d_cost_MSE = grad(cost_MSE, 2)


def cost_CE(X,y,theta, lmb=0):
    pass

d_cost_CE = grad(cost_CE, 2)

# Learning scheduler inspired my ~~Morten~~ SKlearn

In [ ]:
# Time based learning-rate scheduler, smaller steps as time moves forward
def learning_scheduler(t, alpha = 1):
    return 1./(alpha * (t + t0))

def step_length(t, t0, t1):
    return t0/(t+t1)


# SGD

In [ ]:
def new_sgd(X_train, t_train, theta, n_epoch, batch_size, eta, lr_scheduler=False, lmb=0):
    n_batches = int(X_train.shape[0] // batch_size)
    Xt = np.concatenate((X_train, t_train), axis=1)
    print(f"Number of minibatches: {n_batches}")
    
    if lr_scheduler:
        t0 = 1.; t1 = 100
        eta = t0/t1
        
    print(f"Using learning rate scheduler with initial learning rate: {eta}")

    
    for epoch in tqdm(range(n_epoch), f"Training {n_epoch} epochs"):      
        batches = np.take(Xt, np.random.permutation(Xt.shape[0]), axis=0)
        batches = np.array_split(batches, n_batches, axis=0)
        batch_counter = 0
        
        for batch in batches:
            xi = batch[:, :-1]
            yi = batch[:, -1].reshape(-1,1)

            gradients = (2./xi.shape[0])*d_cost_MSE(xi, yi, theta, lmb)
            theta = theta - eta*gradients

            if lr_scheduler:
                t = epoch*n_batches+epoch
                eta = step_length(t, t0, t1)
                #eta = learning_scheduler(eta, epoch*batch_counter)
            
            batch_counter += 1
            
    return theta.ravel()
   


# Morten SGD

In [ ]:

def hands_sgd(X_train, t_train, theta, n_epoch, batch_size, eta, lr_scheduler=False, lmb=0):
    n_batches = int(X_train.shape[0] / batch_size)
    print(f"Number of minibatches: {n_batches}")
    
    t0 = 1.; t1 = 100
        
    print(f"Using learning rate scheduler with initial learning rate: {eta}")
    for epoch in tqdm(range(n_epoch), f"Training {n_epoch} epochs"):
        for i in range(n_batches):
            random_idx = batch_size*np.random.randint(n_batches)
            xi = X_train[random_idx:random_idx+batch_size]
            yi = t_train[random_idx:random_idx+batch_size]

            gradients = (2./batch_size)*d_cost_MSE(xi, yi, theta, lmb)
            eta = step_length(epoch*batch_size+i, t0, t1)
            theta = theta - eta*gradients
            
    return theta.ravel()

# Gard SGD

In [ ]:
def gard_sgd(X_train, t_train, theta, n_epoch, batch_size, eta, lr_scheduler=False, lmb=0):
    n_batches = np.ceil(X_train.shape[0] / batch_size)
    print(f"Number of minibatches: {n_batches}")
    
    if lr_scheduler:
        t0 = 1.; t1 = 100
        eta = t0/t1
        
    print(f"Using learning rate scheduler with initial learning rate: {eta}")

    indicies = np.arange(X_train.shape[0])
    
    for epoch in range(n_epoch):
        np.random.shuffle(indicies)
        minibatches_idx = np.array_split(indicies, n_batches)
        
        for minibatch in range(len(minibatches_idx)):
            
            
            xi = np.take(X_train, minibatches_idx[minibatch],axis=0)
            yi = np.take(t_train, minibatches_idx[minibatch],axis=0).reshape(-1,1)

            gradients = (2./xi.shape[0])*d_cost_MSE(xi, yi, theta, lmb)
            theta = theta - eta*gradients

            if lr_scheduler:
                t = epoch*n_batches+epoch
                eta = step_length(t, t0, t1)
                #eta = learning_scheduler(eta, epoch*batch_counter)
            
            
    return theta.ravel()

In [ ]:
"""
SEED_VALUE = 70707070
np.random.seed(SEED_VALUE)

n = 100
x = 2*np.random.rand(n,1)
t = 4+3*x+np.random.randn(n,1)

X = np.c_[np.ones((n,1)), x]
X = np.hstack([X, x**2])
X = X[:,1:]

X_train, X_test, t_train, t_test = train_test_split(X, t, test_size=0.2, shuffle=True)

X_train, X_test = standard_scaling(X_train, X_test)
t_train, t_test = standard_scaling(t_train, t_test)

_,features_X = X_train.shape 
theta_initial_values = np.random.randn(features_X,1)
eta = 0.01

n_epochs = 100
batch_size = 5  #size of each minibatch
lr_scheduler = True

theta = hands_sgd(X_train, t_train, theta_initial_values, n_epochs, batch_size, eta, lr_scheduler=True)
print(f"theta from new SGD: {theta}")
#print(f"Predicted value our SGD: {X_test @ theta}")
print(f"MSE for SGD: {MSE(t_test, X_test @ theta)}")
"""

# Momentum SGD

In [ ]:
def momentum_sgd(X_train, t_train, theta, n_epoch, batch_size, eta, beta, lr_scheduler=False, ridge=False, lmb=0):
    n_batches = int(X_train.shape[0] // batch_size)
    Xt = np.concatenate((X_train, t_train), axis=1)
    print(f"Number of minibatches: {n_batches}")
    
    if lr_scheduler:
        t0 = 1.0; t1 = 100
        eta = t0/t1
        print(f"Using learning rate scheduler with initial learning rate: {eta}")

    
    for epoch in tqdm(range(n_epoch), f"Training {n_epoch} epochs"):      
        batches = np.take(Xt, np.random.permutation(Xt.shape[0]), axis=0)
        batches = np.array_split(batches, n_batches, axis=0)
        momentum = 0
        
        for batch in batches:
            xi = batch[:, :-1]
            yi = batch[:, -1].reshape(-1,1)
            
            # (2.0/n)*X.T @ (X @ beta-y)
            gradients = 2.0 * xi.T @ ((xi @ theta)-yi)
            """
            RIDGE NOT YET SUPPORTED
            if ridge:
                # TODO: the coff regularization is not implemented correct. 
                #gradients +=  lmb*np.eye(theta.shape[0])
                update = lmb*np.ones(theta.shape[0]).reshape((-1,1))
                gradients += update 
            """
            momentum = momentum*beta - eta*gradients
            theta = theta + momentum

            if lr_scheduler:
                t = epoch*n_batches+epoch
                eta = step_length(t, t0, t1)
            
    return theta.ravel()

# RMS prop

In [ ]:
def rmsprop(X_train, t_train, theta, n_epoch, batch_size, eta, beta, eps, lr_scheduler=False, ridge=False, lmb=0):
    n_batches = int(X_train.shape[0] // batch_size)
    Xt = np.concatenate((X_train, t_train), axis=1)
    print(f"Number of minibatches: {n_batches}")
    
    if lr_scheduler:
        t0 = 1.0; t1 = 100
        eta = t0/t1
        print(f"Using learning rate scheduler with initial learning rate: {eta}")

    
    for epoch in tqdm(range(n_epoch), f"Training {n_epoch} epochs"):      
        batches = np.take(Xt, np.random.permutation(Xt.shape[0]), axis=0)
        batches = np.array_split(batches, n_batches, axis=0)
        
        s = np.zeros((X_train.shape[-1],1))
        
        for batch in batches:
            xi = batch[:, :-1]
            yi = batch[:, -1].reshape(-1,1)
            
            # (2.0/n)*X.T @ (X @ beta-y)
            g = 2.0* xi.T @ ((xi @ theta)-yi)
            """
            RIDGE NOT YET SUPPORTED
            if ridge:
                # TODO: the coff regularization is not implemented correct. 
                #gradients +=  lmb*np.eye(theta.shape[0])
                update = lmb*np.ones(theta.shape[0]).reshape((-1,1))
                gradients += update 
            """
            s = s*beta + (1 - beta)*np.power(g, 2)
            theta = theta - eta*(g/np.sqrt(s + eps))

            if lr_scheduler:
                t = epoch*n_batches+epoch
                eta = step_length(t, t0, t1)
            
    return theta.ravel()

# ADAM

In [ ]:
def adam(X_train, t_train, theta, n_epoch, batch_size, eta, beta1, beta2, eps, lr_scheduler=False, ridge=False, lmb=0):
    n_batches = int(X_train.shape[0] // batch_size)
    Xt = np.concatenate((X_train, t_train), axis=1)
    print(f"Number of minibatches: {n_batches}")
    
    if lr_scheduler:
        t0 = 1.0; t1 = 100
        eta = t0/t1
        print(f"Using learning rate scheduler with initial learning rate: {eta}")

    time = 0
    for epoch in tqdm(range(n_epoch), f"Training {n_epoch} epochs"):      
        batches = np.take(Xt, np.random.permutation(Xt.shape[0]), axis=0)
        batches = np.array_split(batches, n_batches, axis=0)
        
        s = np.zeros((X_train.shape[-1],1))
        m = np.zeros_like(s)
        
        for batch in batches:
            time += 1

            xi = batch[:, :-1]
            yi = batch[:, -1].reshape(-1,1)
            
            # (2.0/n)*X.T @ (X @ beta-y)
            g = 2.0* xi.T @ ((xi @ theta)-yi)
            """
            RIDGE NOT YET SUPPORTED
            if ridge:
                # TODO: the coff regularization is not implemented correct. 
                #gradients +=  lmb*np.eye(theta.shape[0])
                update = lmb*np.ones(theta.shape[0]).reshape((-1,1))
                gradients += update 
            """
            m = beta1*m + (1 - beta1)*g # First moment
            s = beta2*s + (1 - beta2)*np.power(g, 2) # Second moment

            m = m / (1 - np.power(beta1, time))
            s = s / (1 - np.power(beta2, time))

            theta = theta - eta * (m / (np.sqrt(s) + eps))

            if lr_scheduler:
                t = epoch*n_batches+epoch
                eta = step_length(t, t0, t1)
            
    return theta.ravel()

# Real Terrain data

In [ ]:
np.random.seed(4155)

degree = 10

X = create_X(x,y, degree)
X = X[:,1:]
X_train, X_test, t_train, t_test = train_test_split(X, z_flat, test_size=0.2, shuffle=True)

print(f"X has {X.shape[1]} features")

X_train, X_test = standard_scaling(X_train, X_test)
t_train, t_test = standard_scaling(t_train, t_test)

XT_X = X_train.T @ X_train
H = (2./X_train.shape[0]) * XT_X
print(f"Upper limit learing rate: {2./np.max(np.linalg.eig(H)[0])}")

theta_initial_values = np.random.randn(X_train.shape[1],1)

theta = new_sgd(X_train, t_train, theta_initial_values, 100, 5, eta=1./np.max(np.linalg.eig(H)[0]), lmb=0, lr_scheduler=True)
print(f"MSE for SGD: {MSE(t_test, X_test @ theta)}")

theta = hands_sgd(X_train, t_train, theta_initial_values, 100, 5, eta=1./np.max(np.linalg.eig(H)[0]), lr_scheduler=True)
print(f"MSE for Morten SGD: {MSE(t_test, X_test @ theta)}")

theta = gard_sgd(X_train, t_train, theta_initial_values, 100, 5, eta=1./np.max(np.linalg.eig(H)[0]), lr_scheduler=True)
print(f"MSE for Gard SGD: {MSE(t_test, X_test @ theta)}")

sgdreg = SGDRegressor(max_iter = 100, fit_intercept=False, penalty='l2', eta0=0.01, alpha=1, learning_rate='optimal')
sgdreg.fit(X_train,t_train.ravel())
print(f"MSE for SKlearn: {MSE(t_test, X_test @ sgdreg.coef_)}")

beta_hat = np.linalg.pinv(XT_X) @ (X_train.T @ t_train)
#print(f"beta from inversion: {beta_hat}")
print(f"MSE for OLS: {MSE(t_test, X_test @ beta_hat)}")

# Testing without regularizer

In [ ]:
# Created test data
""""""
SEED_VALUE = 70707070
np.random.seed(SEED_VALUE)

n = 100
x = 2*np.random.rand(n,1)
t = 4+3*x+np.random.randn(n,1)

X = np.c_[np.ones((n,1)), x]
X = np.hstack([X, x**2])
X = X[:,1:]

X_train, X_test, t_train, t_test = train_test_split(X, t, test_size=0.2, shuffle=True)

# Terrain data
"""
degree = 1
X = create_X(x,y, n=degree)
X = remove_intercept(X)
X_train, X_test, t_train, t_test = train_test_split(X, z_flat, test_size=0.2, shuffle=True)
"""

X_train, X_test = standard_scaling(X_train, X_test)
t_train, t_test = standard_scaling(t_train, t_test)

XT_X = X_train.T @ X_train
H = (2./n) * XT_X
print(f"Upper limit learing rate: {2./np.max(np.linalg.eig(H)[0])}")

_,features_X = X_train.shape 
theta_initial_values = np.random.randn(features_X,1)
eta = 0.01
lmb=0.0001

n_epochs = 100
batch_size = 5  #size of each minibatch
lr_scheduler = True

theta = new_sgd(X_train, t_train, theta_initial_values, n_epochs, batch_size, eta, lr_scheduler=True)
print(f"theta from SGD: {theta}")
#print(f"Predicted value our SGD: {X_test @ theta}")
print(f"MSE for SGD: {MSE(t_test, X_test @ theta)}")

theta = hands_sgd(X_train, t_train, theta_initial_values, n_epochs, batch_size, eta, lr_scheduler=True)
print(f"theta from shuffle SGD: {theta}")
#print(f"Predicted value our SGD: {X_test @ theta}")
print(f"MSE for shuffle SGD: {MSE(t_test, X_test @ theta)}")

"""
theta = momentum_sgd(X_train, t_train, theta_initial_values, n_epochs, batch_size, eta, beta=0.9, lr_scheduler=lr_scheduler)
print(f"theta from momentum SGD: {theta}")
#print(f"Predicted value our momentum SGD: {X_test @ theta}")
print(f"MSE for momentum SGD: {MSE(t_test, X_test @ theta)}")


theta = rmsprop(X_train, t_train, theta_initial_values, n_epochs, batch_size, eta, beta=0.9, eps=10**(-8), lr_scheduler=lr_scheduler)
print(f"theta from RMSprop: {theta}")
print(f"MSE for RMSprop: {MSE(t_test, X_test @ theta)}")

theta = adam(X_train, t_train, theta_initial_values, n_epochs, batch_size, eta, beta1=0.9, beta2=0.99, eps=10**(-8), lr_scheduler=lr_scheduler)
print(f"theta from ADAM: {theta}")
print(f"MSE for ADAM: {MSE(t_test, X_test @ theta)}")
"""

beta_hat = np.linalg.pinv(XT_X) @ (X_train.T @ t_train)
print(f"beta from inversion: {beta_hat}")
print(f"MSE for OLS: {MSE(t_test, X_test @ beta_hat)}")

sgdreg = SGDRegressor(max_iter = n_epochs, fit_intercept=False, penalty=None, eta0=eta, alpha=1, learning_rate='optimal')
# sgdreg = SGDRegressor(max_iter = n_epochs, fit_intercept=False, penalty=None, eta0=eta)
sgdreg.fit(X_train,t_train.ravel())
print(f"sgdreg from scikit: {sgdreg.coef_}")
#print(f"Predicted value  SciKit: {X_test @ sgdreg.coef_}")
print(f"MSE for SKlearn: {MSE(t_test, X_test @ sgdreg.coef_)}")

# print(f"sgdreg from scikit: {sgdreg.intercept_}, {sgdreg.coef_}")

# Testing L2 regularizer

In [ ]:
from sklearn.linear_model import SGDRegressor

# Created test data
""""""
SEED_VALUE = 70707070
#np.random.seed(SEED_VALUE)

n = 100
x = 2*np.random.rand(n,1)
t = 4+3*x+np.random.randn(n,1)
X = np.c_[np.ones((n,1)), x]
X = np.hstack([X, x**2])
X = X[:,1:]
X_train, X_test, t_train, t_test = train_test_split(X, t, test_size=0.2, shuffle=True)


# Terrain data
"""
degree = 1
X = create_X(x,y, n=degree)
X = remove_intercept(X)
X_train, X_test, t_train, t_test = train_test_split(X, z_flat, test_size=0.2, shuffle=True)
"""

X_train, X_test = standard_scaling(X_train, X_test)
t_train, t_test = standard_scaling(t_train, t_test)

_,features_X = X_train.shape 
theta_initial_values = np.random.randn(features_X,1)
eta = 0.01
lmb=0.01

n_epochs = 1000
batch_size = 5  #size of each minibatch
lr_scheduler = True

theta = new_sgd(X_train, t_train, theta_initial_values, n_epochs, batch_size, eta, lr_scheduler=lr_scheduler, ridge=True, lmb=lmb)

print(f"theta from new SGD: {theta}")
print(f"MSE for SGD with l2 regularization: {MSE(t_test, X_test @ theta)}")


#theta = momentum_sgd(X_train, t_train, theta_initial_values, n_epochs, batch_size, eta, beta=0.9, lr_scheduler=lr_scheduler)

#print(f"theta from momentum SGD: {theta}")

sgdreg = SGDRegressor(max_iter = n_epochs, fit_intercept=False, penalty='l2', eta0=eta, alpha=lmb, learning_rate='optimal')
# sgdreg = SGDRegressor(max_iter = n_epochs, fit_intercept=False, penalty=None, eta0=eta)
sgdreg.fit(X_train,t_train.ravel())
print(f"sgdreg from scikit: {sgdreg.coef_}")
print(f"MSE for SKlearn with l2 regularization: {MSE(t_test, X_test @ sgdreg.coef_)}")

# print(f"sgdreg from scikit: {sgdreg.intercept_}, {sgdreg.coef_}")